In [1]:
import numpy as np
import pprint as pp

from os import listdir
from os.path import isfile, join
from datetime import datetime

import os
import xml.etree.ElementTree as ET
from sklearn.metrics import average_precision_score
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.transforms import FiveCrop, ToTensor, Lambda, Compose, CenterCrop, Normalize

import matplotlib.pyplot as plt

from utils import PascalVOCLabelLoader, PascalVOCDataset, ImageOnlyDataset


In [2]:
def train_epoch(model, device, train_loader, optimizer, num_epochs, val_loader, loss_fun, class_list):
    train_losses = []
    train_ap_scores = []
    val_losses = []
    val_ap_scores = []
    
    now = datetime.now()
    directory = 'model_' + now.strftime("%m%d%Y_%H%M%S")
    if not os.path.exists(directory):
        os.makedirs(directory)

    for epoch in range(1, num_epochs + 1):
        print(f'\nEpoch: {epoch}')
        train_loss, train_ap_score = train(model, device, train_loader, optimizer, epoch, loss_fun, class_list)
        val_loss, val_ap_score = validate(model, device, val_loader, epoch, loss_fun, class_list)

        if (len(val_losses) > 0) and (val_loss < min(val_losses)):
            torch.save(model.state_dict(), directory + f'/resnet18_model_{epoch}.pt')
            print(f"Saving model (epoch {epoch}) with lowest validation loss: {val_loss}")

        train_losses.append(train_loss)
        train_ap_scores.append(train_ap_score)
        val_losses.append(val_loss)
        val_ap_scores.append(val_ap_score)

    print("Training and validation complete.")
    return train_losses, train_ap_scores, val_losses, val_ap_scores

In [3]:
def train(model, device, train_loader, optimizer, epoch, loss_fun, class_list):
    model.train()
    train_losses = []
    ap_score = 0
    
    for idx, batch in enumerate(train_loader):
        data = batch[0].to(device)
        target = batch[1].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fun(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        pred = torch.sigmoid(output)
        
        if idx == 0:
            predictions = pred
            targets = target
        else:
            predictions = torch.cat((predictions, pred))
            targets = torch.cat((targets, target))
        if idx % 100 == 0:
            print(f'Epoch: {epoch}, Training_Samples: {idx}/{len(train_loader)}, Loss: {loss.item()},\
            Total AP: {ap(targets, predictions, len(class_list)):.4f}')
    
    train_loss = torch.mean(torch.tensor(train_losses))
    total_ap_score = ap(targets, predictions, len(class_list))
    classwise_ap_scores = classwise_ap(targets, predictions, class_list)
    mean_ap_score = np.mean([ 0 if np.isnan(i) else i for i in list(classwise_ap_scores.values())])
    
    print(f'Training set: Average loss: {train_loss:.4f}, Total AP: {total_ap_score:.4f}')
    print('Validation set: Classwise AP:')
    pp.pprint(classwise_ap_scores)
    print(f'Validation set: Mean AP: {mean_ap_score:.4f})')
    return train_loss, mean_ap_score

In [4]:
def validate(model, device, val_loader, epoch, loss_fun, class_list):
    model.eval()
    val_loss = 0
    ap_score = 0
    
    with torch.no_grad():
        for idx, batch in enumerate(val_loader):
            data = batch[0].to(device)
            target = batch[1].to(device)
            output = model(data)
            
            # compute the batch loss
            batch_loss = loss_fun(output, target).item()
            val_loss += batch_loss
            pred = torch.sigmoid(output)
            
            if idx == 0:
                predictions = pred
                targets = target
            else:
                predictions = torch.cat((predictions, pred))
                targets = torch.cat((targets, target))
            if idx % 100 == 0:
                print(f'Epoch: {epoch}, Validation_Samples: {idx}/{len(val_loader)}, Loss: {batch_loss},\
                Total AP: {ap(targets, predictions, len(class_list)):.4f}')
                

    val_loss /= len(val_loader)
    total_ap_score = ap(targets, predictions, len(class_list))
    classwise_ap_scores = classwise_ap(targets, predictions, class_list)
    mean_ap_score = np.mean([ 0 if np.isnan(i) else i for i in list(classwise_ap_scores.values())])
    
    print(f'Validation set: Average loss: {val_loss:.4f}, Total AP: {total_ap_score:.4f}')
    print('Validation set: Classwise AP:')
    pp.pprint(classwise_ap_scores)
    print(f'Validation set: Mean AP: {mean_ap_score:.4f})')
    
    return val_loss, mean_ap_score

In [5]:
def test(model, device, test_loader, loss_fun, class_list):
    model.eval()
    test_loss = 0
    ap_score = 0
    
    with torch.no_grad():
        for idx, batch in enumerate(test_loader):
            data = batch[0].to(device)
            target = batch[1].to(device)
            output = model(data)
            
            # compute the batch loss
            batch_loss = loss_fun(output, target).item()
            test_loss += batch_loss
            pred = torch.sigmoid(output)
            
            if idx == 0:
                predictions = pred
                targets = target
                image_paths = list(batch[2])
            else:
                predictions = torch.cat((predictions, pred))
                targets = torch.cat((targets, target))
                image_paths += list(batch[2])
            if idx % 100 == 0:
                print(f'Test_Samples: {idx}/{len(test_loader)}, Loss: {batch_loss},\
                Total AP: {ap(targets, predictions, len(class_list)):.4f}')
                

    test_loss /= len(test_loader)
    total_ap_score = ap(targets, predictions, len(class_list))
    classwise_ap_scores = classwise_ap(targets, predictions, class_list)
    mean_ap_score = np.mean([ 0 if np.isnan(i) else i for i in list(classwise_ap_scores.values())])
    
    print(f'Test set: Average loss: {test_loss:.4f}, Total AP: {total_ap_score:.4f}')
    print('Test set: Classwise AP:')
    pp.pprint(classwise_ap_scores)
    print(f'Test set: Mean AP: {mean_ap_score:.4f})')
    
    return predictions, targets, image_paths

In [6]:
def ap(targets, predictions, num_classes):
    return average_precision_score(targets.reshape(-1, num_classes).cpu(), predictions.reshape(-1, num_classes).cpu())

In [7]:
def classwise_ap(targets, predictions, class_list):
    classwise_ap_dic = {}
    for i, label in enumerate(class_list):
        classwise_ap_dic[label] = average_precision_score(targets[:, i].cpu(),  predictions[:, i].cpu())
    return classwise_ap_dic

In [8]:
def top_bot_scores_50(img_names, scores, class_num, top=50):
    """
    img_names: a list containing the image names
    scores: a list containing the predicted scores for each class
    class_num: the column of the class
    num_ins: number of instances you want
    
    returns: a list containing the top and bottom 50 scores and images of this class in the format
             [top_50_scores, top_50_images, bot_50_scores, bot_50_images]
    
    """

    # Use the scores to get the new order in a list, from highest to lowest
    n_scores = np.copy(scores.cpu())
    n_img_names = np.copy(img_names)
    
    reorder = n_scores[:,class_num].argsort()[::-1]
    # Reorder the img_names
    r_img_names = n_img_names[reorder]
    top_img = r_img_names[:top]
    bottom_img = r_img_names[-top:]
    
    return top_img, bottom_img, reorder

In [9]:
def img_grid_plotter(dataloader, class_name, num_img, rank_name):
    real_batch = next(iter(dataloader))
    plt.figure(figsize=(20,10))
    plt.axis("off")
    plt.title(f'Top {num_img} {rank_name} scored images for class {class_name}')
    plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:50], padding=2, nrow=10, normalize=True).cpu(),(1,2,0)))

In [10]:
# Create dataloaders for top and bot images
def plot_top_images(cls_img_pairs, img_path, tr, class_list, rank_name):
    """
    cls_img_pairs: a list containing the pairs of a class with its respective top or bottom x image names
    img_path: image path
    tr: your transform sequence
    class_list: a list mapping for the class labels
    rank_name: determine if the set is either highest or lowest
    
    returns Nothing, but plots out the random 5 classes and their top/bottom x images
    """
    for x in cls_img_pairs:
        num_img = len(x[1])
        # Create the dataset and subsequently dataloader
        i_dataset = ImageOnlyDataset(img_path, x[1], transform=tr)
        i_dataloder = DataLoader(i_dataset, batch_size=num_img, shuffle=False)
        
        
        # Use plotter function to print a grid of the images
        img_grid_plotter(i_dataloder, class_list[x[0]], num_img, rank_name)
    
    

In [11]:
def get_tail_acc(sorted_predictions, sorted_targets, t_min, t_max, t_num, t_vals):
    tail_acc_list = []
    for t_val in t_vals:
        tp = 0
        fp = 0
        for (prediction, target) in zip(sorted_predictions, sorted_targets):
            pred = 0
            if prediction > t_val:
                pred = 1
                if pred == target:
                    tp += 1
                else:
                    fp += 1
        tail_acc = 0
        if tp+fp > 0:
            tail_acc = tp/(tp+fp)
        tail_acc_list.append(tail_acc)
        
    return tail_acc_list

In [12]:
def run():

    ###### Global Variables (Change it if you want) ######
    class_list = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 
                  'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')    
    lr = 1e-3
    momentum = 0.9
    imgpath = 'data/VOCdevkit/VOC2012/JPEGImages/'
    voc_path = 'data/VOCdevkit/VOC2012'
    rand_crop_size = 300
    bs = 16 # Batch size not bullshit
    epochs = 5

    tr = transforms.Compose([transforms.RandomResizedCrop(rand_crop_size),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.4589, 0.4355, 0.4032],[0.2239, 0.2186, 0.2206])])

    ###### Create datasets and respective dataloaders ######
    pvc = PascalVOCLabelLoader(voc_path)
    
    train_dataset = PascalVOCDataset(imgpath, class_list, pvc, 'train', transform=tr)
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

    val_dataset = PascalVOCDataset(imgpath, class_list, pvc, 'trainval', transform=tr)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=True)

    test_dataset = PascalVOCDataset(imgpath, class_list, pvc, 'val', transform=tr)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

    ###### Model Parameters ######
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(512, len(class_list))
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    loss_fun = nn.BCEWithLogitsLoss()

    ###### Start Taining ######
    start = time.time()
    train_losses, train_ap_scores, val_losses, val_ap_scores = train_epoch(model, device, train_loader, optimizer, epochs, val_loader, loss_fun, class_list)
    print(f'Total training taken: {time.time() - start}')

    ###### Plot Loss and mean AP score of training and validation dataset ######
    plt.figure(figsize=(20,10))
    plt.subplot(121),
    plt.plot(train_losses, label = "Training Loss"),
    plt.plot(val_losses, label = "Validation Loss"),
    plt.title('Loss'),
    plt.legend(frameon = False)
    plt.subplot(122),
    plt.plot(train_ap_scores, label = "Training Mean AP Score"),
    plt.plot(val_ap_scores, label = "Validation Mean AP Score"),
    plt.title('Mean AP Score'),
    plt.legend(frameon = False)
    plt.show()

    ###### Run the test function and get the respective predictions, targets and impage paths ######
    test_predictions, test_targets, test_image_paths = test(model, device, test_loader, loss_fun, class_list)

    # Randomly pick 5 classes
    np.random.seed(10)
    random_classes = np.random.choice(range(len(class_list)), 5, replace=False)

    # Each of these contains 5 lists, that contain [class_number, [array of image_names]]
    t_images_all = []
    b_images_all = []
    
    # Store the sorted list of images based on scores with their respective class
    for x in random_classes:
        t_images, b_images, _ = top_bot_scores_50(test_image_paths, test_predictions, x)
        t_images_all.append([x, t_images])
        b_images_all.append([x, b_images])

    ###### Plot the top 50 highest and lowest images ######
    # Display top 50 images random 5 classes
    plot_top_images(t_images_all, imgpath, tr, class_list, "highest")

    # Display bottom 50 images random 5 classes
    plot_top_images(b_images_all, imgpath, tr, class_list, "lowest")

    # Define variables for calculating tail accuracies
    tail_acc_classwise = []
    t_max = np.max(test_predictions.numpy())
    t_min=0.5
    t_num=20
    t_vals = np.linspace(t_min, t_max, t_num, endpoint=False)
    
    ###### Calculate Tail Accuracy for all classes ######
    for i in range(len(class_list)):
        _, _, reorder = top_bot_scores_50(test_image_paths, test_predictions, i)
        sorted_predictions = np.copy(test_predictions.cpu())[reorder]
        sorted_predictions = sorted_predictions[:50, i]
        sorted_targets = np.copy(test_targets.cpu())[reorder]
        sorted_targets = sorted_targets[:50, i]

        tail_acc_classwise.append(get_tail_acc(sorted_predictions, sorted_targets, t_min, t_max, t_num, t_vals))

    ###### Plot average tail accuracy of all classes ######
    avg_tail_acc = np.mean(tail_acc_classwise, axis=0)
    plt.figure(figsize=(20,10))
    plt.plot(t_vals, avg_tail_acc, 'b', t_vals, avg_tail_acc, 'ro')
    plt.xticks(t_vals)
    plt.title('Average of classwise tail accuraries over top-50 ranked images in each class')
    plt.ylabel('Tail Accuracy')
    plt.xlabel('t value')
    plt.show()

In [13]:
if __name__=='__main__':
    run()


Epoch: 1


RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.